In [115]:
import sys 
sys.path.append('../..')
from cox.utils import Parameters
from cox.store import Store
from cox.readers import CollectionReader
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import itertools
import numpy as np
import torch as ch
from torch import Tensor
import torch.nn as nn
from torch.distributions import Gumbel, Uniform
from torch.distributions.multivariate_normal import MultivariateNormal
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
from delphi.utils import constants as consts
from delphi import oracle
from delphi import train
from delphi.utils import defaults
import datetime

# Store Information, etc

In [2]:
STORE_PATH = '/home/pstefanou/MultinomialLogisticRegression'
STORE_TABLE_NAME = 'multi_log_reg'

store = Store(STORE_PATH)

store.add_table(STORE_TABLE_NAME, { 
    'delphi_cos': float,
    'mle_cos': float,
    'alpha': float, 
    'C': float,
})

Logging in: /home/pstefanou/LogisticRegression/5b31e98e-45d5-4775-b2ac-635e2e34fd3f


# Standard Gumbel Distribution

(array([  3.,   2.,   1.,   7.,  10.,  18.,  46.,  67.,  95., 131., 183.,
        232., 238., 323., 314., 403., 413., 424., 448., 426., 442., 463.,
        408., 413., 389., 376., 353., 335., 261., 266., 235., 247., 188.,
        180., 173., 162., 138., 126., 108.,  86.,  98.,  91.,  66.,  62.,
         73.,  38.,  50.,  38.,  33.,  29.,  26.,  25.,  24.,  28.,  17.,
         14.,  20.,  13.,  13.,  13.,  11.,   9.,   9.,   6.,  11.,   5.,
          7.,   2.,   4.,   3.,   2.,   2.,   2.,   2.,   3.,   2.,   3.,
          2.,   1.,   3.,   1.,   0.,   0.,   0.,   0.,   2.,   1.,   0.,
          1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,
          1.]),
 array([-2.3604083 , -2.2408545 , -2.121301  , -2.0017471 , -1.8821934 ,
        -1.7626398 , -1.6430861 , -1.5235324 , -1.4039787 , -1.284425  ,
        -1.1648712 , -1.0453175 , -0.92576385, -0.80621016, -0.6866565 ,
        -0.56710273, -0.44754905, -0.32799533, -0.20844163, -0.08888792,
         0.03066578,  0.15

In [116]:
G = Gumbel(0, 1)

samples = G.sample(ch.Size([10000, 1])).T
plt.hist(samples, bins=100)

## Default Experiment Parameters

In [138]:
# procedure hyperparameters
args = Parameters({ 
    'epochs': 25,
    'num_workers': 0, 
    'batch_size': 10,
    'bias': True,
    'num_samples': 100,
    'clamp': True, 
    'radius': 5.0, 
    'var_lr': 1e-2,
    'lr': 1e-1,
    'shuffle': False, 
    'tol': 1e-2,
    'custom_lr_multiplier': consts.COSINE,
    'samples': 10000,  # number of samples to generate for ground truth
    'in_features': 1, # number of in-features to multi-log-reg
    'k': 2, # number of classes
    'lower': -1, # lower bound for generating ground truth weights
    'upper': 1,  # upper bound for generating ground truth weights
    'custom_criterion': F.gumbel_softmax,
})

# args = defaults.check_and_fill_args(train_kwargs, defaults.TRAINING_ARGS, CIFAR)

# Multinomial Logistic Regression

In [172]:
# generate ground-truth from uniform distribution
U = Uniform(args.lower, args.upper)
ground_truth = nn.Linear(in_features=args.IN_FEATURES, out_features=args.K, bias=args.bias)
ground_truth.weight = nn.Parameter(U.sample(ch.Size([args.K, args.IN_FEATURES])))
if ground_truth.bias is not None: 
    ground_truth.bias = nn.Parameter(U.sample(ch.Size([args.K,])))
print("ground truth weights: {}".format(ground_truth.weight))
print("ground truth bias: {}".format(ground_truth.bias))
# independent variable 
M = MultivariateNormal(ch.zeros(args.IN_FEATURES), ch.eye(args.IN_FEATURES)/args.IN_FEATURES)
X = M.sample(ch.Size([args.samples]))

# determine base model logits 
z = ground_truth(X)
# add gumbel noise to all logits and take argmax
y = ch.argmax(z + G.sample(z.size()), dim=1)
# print("accuracy: {}".format((ch.argmax(ch.log_softmax(ground_truth(X), dim=1), dim=1) == y).nonzero(as_tuple=False).flatten().size(0)/args.samples))
print("accuracy: {}".format((F.gumbel_softmax(ch.log_softmax(ground_truth(X), dim=1), dim=1) == y).nonzero(as_tuple=False).flatten().size(0)/args.samples))

ground truth weights: Parameter containing:
tensor([[-0.4753],
        [-0.3206]], requires_grad=True)
ground truth bias: Parameter containing:
tensor([-0.0283,  0.6675], requires_grad=True)


RuntimeError: The size of tensor a (2) must match the size of tensor b (10000) at non-singleton dimension 1

# Model to Generate Data

10000

In [140]:
ds = TensorDataset(X, y)
loaders = (DataLoader(ds), None)

In [113]:
results = train.train_model(args, model=nn.Linear(in_features=args.IN_FEATURES, out_features=args.k, bias=args.bias), loaders=loaders)

TypeError: '<' not supported between instances of 'NoneType' and 'float'

# Gradient

In [5]:
# perform each experiment a total of 10 times
for iter_ in range(100): 
    for C in [-2, -.5, -1, -.5]:
        PREDICATE = Left(Tensor([C]))
        args.__setattr__('phi', PREDICATE)
        # generate random uniform weights
        W = Uniform(LOWER, UPPER).sample(ch.Size([DIMS, 1]))
        W0 = Uniform(LOWER, UPPER).sample(ch.Size([1, 1]))

        # generate data
        X = Uniform(-100, 100).sample(ch.Size([NUM_SAMPLES, DIMS]))
        z = X.mm(W) + W0 + logistic.sample(ch.Size([NUM_SAMPLES, 1]))
        y = ch.where(z > 0, 1.0, 0.0)
        # truncate
        indices = PREDICATE(z).nonzero(as_tuple=False).flatten()
        z_trunc, y_trunc, x_trunc = z[indices], y[indices], X[indices]

        args.__setattr__('alpha', Tensor([y_trunc.size(0) / NUM_SAMPLES]))

        # truncated logistic regression
        data = TruncatedLogisticRegressionDataset(x_trunc, y_trunc, bias=args.bias)
        S = DataLoader(data, batch_size=args.batch_size, num_workers=args.num_workers, shuffle=args.shuffle)
        trunc_log_reg = truncated_logistic_regression(phi=args.phi, alpha=args.alpha, bias=args.bias, epochs=args.epochs, 
                                                      lr=args.lr, num_samples=args.num_samples, tol=args.tol)
        results = trunc_log_reg.fit(S)
        w, w0 = results.weight.detach().cpu(), results.bias.detach().cpu()

        # parameter estimates 
        real_params = ch.cat([W.T, W0], axis=1)
        mle_params = ch.cat([data.w, data.w0.unsqueeze(0)], axis=1)
        delphi_params = ch.cat([w, w0.unsqueeze(0)], axis=1)

        # metrics
        mle_cos = cosine_similarity(mle_params, real_params)
        delphi_cos = cosine_similarity(delphi_params, real_params)

        store[STORE_TABLE_NAME].append_row({ 
            'delphi_cos': delphi_cos,
            'mle_cos': mle_cos,
            'alpha': float(args.alpha),
            'C': C,
        })

# close current store
store.close()

TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'

# Results

In [7]:
reader = CollectionReader(STORE_PATH)
results = reader.df(STORE_TABLE_NAME)
reader.close() # close reader
results.head()

100%|██████████| 8/8 [00:00<00:00, 234.15it/s]

,delphi_cos,mle_cos,alpha,C,exp_id
0,0.755610,0.811039,0.5251,-2.0,408c0688-d0a7-4500-9cfc-8b1ace2fa078
1,0.503841,-0.020497,0.5069,-0.5,408c0688-d0a7-4500-9cfc-8b1ace2fa078
2,0.987580,0.858828,0.5075,-1.0,408c0688-d0a7-4500-9cfc-8b1ace2fa078
3,0.904469,-0.610786,0.5042,-0.5,408c0688-d0a7-4500-9cfc-8b1ace2fa078
4,0.932691,0.932682,0.5145,-2.0,408c0688-d0a7-4500-9cfc-8b1ace2fa078


# All Trials

all trials:


In [8]:
print("all trials:")

# variance vs param mse
sns.lineplot(data=results, x='C', y='delphi_cos', label='delphi', color='blue')
ax = sns.lineplot(data=results, x='C', y='mle_cos', label='mle', color='red')
ax.set(xlabel='Truncation Parameter C', ylabel='Cosine Similarity with Theta*')
plt.show()

# Remove Best and Worst Trial from Results

In [55]:
print("remove best and worst trials:")

# plot results for regression parameter MSE
sns.lineplot(data=results.drop(noise_lambda('unknown_param_mse')), x='var', y='unknown_param_mse', label='unknown', color='blue')
sns.lineplot(data=results.drop(noise_lambda('known_param_mse')), x='var', y='known_param_mse', label='known', color='purple')
ax = sns.lineplot(data=results.drop(noise_lambda('ols_param_mse')), x='var', y='ols_param_mse', label='ols', color='red')
ax.set(xlabel='Noise Variance', ylabel='Squared Error Theta*')
plt.show()

# plot results for regression parameter MSE
sns.lineplot(data=results.drop(noise_lambda('unknown_var_mse')), x='var', y='unknown_var_mse', label='unknown', color='blue')
ax = sns.lineplot(data=results.drop(noise_lambda('ols_var_mse')), x='var', y='ols_var_mse', label='ols', color='red')
ax.set(xlabel='Noise Variance', ylabel='Squared Error Sigma^2*')
plt.show()

remove best and worst trials:


NameError: name 'noise_lambda' is not defined

In [95]:
s = Store('/home/pstefanou/LogisticRegression/', 'ae6627d0-9b20-49ca-9aca-73d0a07637c0')

In [97]:
s.close()

In [147]:
logits = ch.randn(20, 32)
print("soft: {}".format(F.gumbel_softmax(logits, tau=1, hard=False)))
print("hard: {}".format(F.gumbel_softmax(logits, tau=1, hard=True)))

soft: tensor([[7.1953e-03, 1.1171e-01, 2.0755e-03, 1.7464e-01, 4.7869e-03, 2.4271e-02,
         3.1879e-04, 6.7909e-03, 1.5592e-03, 2.2556e-03, 2.2304e-01, 1.2665e-03,
         8.5572e-04, 2.7406e-03, 1.3439e-02, 1.0934e-03, 3.9113e-04, 4.4345e-04,
         1.0770e-03, 2.8287e-03, 1.0578e-02, 4.5601e-03, 9.8678e-04, 1.9285e-03,
         6.0516e-04, 2.5309e-03, 7.6561e-03, 2.2072e-02, 2.7787e-01, 5.1688e-02,
         9.5961e-04, 3.5785e-02],
        [3.1155e-02, 1.3558e-03, 2.6041e-02, 6.8919e-03, 1.3617e-02, 5.4008e-03,
         6.0367e-03, 9.0659e-02, 3.9518e-03, 4.1261e-04, 4.2847e-02, 5.1434e-03,
         2.6073e-01, 1.3171e-02, 1.2959e-02, 4.0272e-02, 6.4154e-03, 1.4522e-02,
         6.1483e-03, 1.3966e-03, 7.5609e-03, 2.5276e-03, 1.5812e-03, 5.8751e-03,
         2.1703e-03, 2.0061e-02, 3.4008e-01, 1.3933e-02, 1.6006e-03, 4.3361e-03,
         7.9422e-03, 3.2001e-03],
        [5.6526e-02, 1.5932e-02, 1.3664e-02, 1.4870e-02, 2.2578e-01, 4.4286e-03,
         4.3574e-03, 6.3728e-03, 9.